# Imports

In [40]:
import pandas as pd
import numpy as np

# Import Data

In [8]:
df = pd.read_csv('transactions_data.csv')

# Create Date Features

Assumption: all datetimes are in the same timezone.

In [9]:
for col in ['transactionDateTime','currentExpDate','accountOpenDate','dateOfLastAddressChange']:
    df[col] = pd.to_datetime(df[col])

In [39]:
df['transactionDate'] = pd.to_datetime(df['transactionDateTime'].dt.date)
df['transactionHour'] = df['transactionDateTime'].dt.hour
df['transactionDayName'] = df['transactionDateTime'].dt.day_name()

In [22]:
df['daysSinceAccountOpen'] = (df['transactionDateTime'] - df['accountOpenDate']).dt.days
df['daysSinceLastAddressChange'] = (df['transactionDateTime'] - df['dateOfLastAddressChange']).dt.days

# CVV Feature

Assuming that the `cardCVV` column contains the actual card CVV and `enteredCVV` contains the CVV that was entered during the transaction.

In [48]:
(df.cardCVV == df.enteredCVV).sum() / len(df)

0.9907510974990419

In [53]:
df['cardCVVMatch'] = (df.cardCVV == df.enteredCVV).astype(int)

In [54]:
df['cardCVVMatch'].value_counts()

1    635977
0      5937
Name: cardCVVMatch, dtype: int64

# Address High Cardinality Features

High cardinality columns contain a high volume of unique categorical values. When these are encoded, they can cause a large increase in the size of the data and we can also run into the curse of dimensionality which means the model will find it harder to identify meaningful patterns and won't be able to generalise well.

Merchant name could be useful as there may be some merchants who see higher volumes of fraud than others. We could use this feature but group the merchants who see very few transactions into an "other" column to avoid high dimensionality. This is quite a basic/naive method and I have seen other methods discussed online called "count encoding" and "catboost encoding" (using CatBoost algorithm) which might be interesting to explore if I had more time.

* https://www.kaggle.com/matleonard/categorical-encodings
* https://greenet09.github.io/datasophy/2019/03/03/Comparing-Methods-for-Dealing-with-High-cardinality-Categorical-Features.html

In [24]:
high_cardinality_cols = [col for col in df.columns if col in df.columns[(df.nunique() >= 20)] 
                        and col in df.select_dtypes('object').columns]
high_cardinality_cols

['merchantName']

In [25]:
df.merchantName.value_counts()[:10]

Lyft            25311
Uber            25263
gap.com         13824
apple.com       13607
target.com      13601
alibaba.com     13583
staples.com     13512
amazon.com      13477
ebay.com        13472
discount.com    13394
Name: merchantName, dtype: int64

In [26]:
df.merchantName.value_counts().min()

1

In [27]:
df.merchantName.value_counts()[:10] / len(df)

Lyft            0.039431
Uber            0.039356
gap.com         0.021536
apple.com       0.021198
target.com      0.021188
alibaba.com     0.021160
staples.com     0.021050
amazon.com      0.020995
ebay.com        0.020987
discount.com    0.020866
Name: merchantName, dtype: float64

Some of these merchant names look like they have specific area codes e.g. Renaissance Hotel #583180 and Renaissance Hotel #499336. With more time, it might be useful to pull out the company name and then use the ID to somehow identify the location to add geospatial features.

I'll categorise any merchants than account for less than 0.05% of the data into an "other" category.

In [28]:
merchant_percent = df.merchantName.value_counts() / len(df)
len(merchant_percent[merchant_percent>0.005])

43

In [29]:
keep_merchants = list(merchant_percent[merchant_percent>0.005].index)

In [32]:
df[df.merchantName=='Renaissance Hotel #499336'].accountNumber.count()

104

In [42]:
df['merchantNameGroup'] = np.where(df.merchantName.isin(keep_merchants),df.merchantName,"other")

In [43]:
df['merchantNameGroup'].nunique()

44

# Create Feature List

Dropping some features (e.g. `posEntryMode`) due to lack of domain knowledge.

In [76]:
features = ['creditLimit', 'availableMoney',
       'transactionAmount',
       'acqCountry', 'merchantCountryCode',
       'merchantCategoryCode', 'transactionType', 
       'currentBalance', 'cardPresent',
       'expirationDateKeyInMatch', 'transactionHour',
       'daysSinceAccountOpen', 'daysSinceLastAddressChange',
       'transactionDayName', 'merchantNameGroup', 'cardCVVMatch']

target = ['isFraud']

# Create Dummy Columns

In [79]:
model_data = pd.get_dummies(df[features + target])

In [80]:
model_data.shape

(641914, 92)

# Save Data

In [81]:
model_data.to_csv('model_data.csv',index=False)